### Connecting country to SQL

In [20]:
#Connecting Country
countries=["Germany","France","Netherlands"]
country_df={"country_name":countries}
country=pd.DataFrame(country_df)
country

,country_name
0,Germany
1,France
2,Netherlands


In [25]:
from your_passwort_file import sql_root_passwort #calling password from the file 

In [27]:
schema = "gans"
host = "127.0.0.1"
user = "root"
password = sql_root_passwort
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'


In [29]:
country.to_sql('country',
                  if_exists='append',
                  con=connection_string,
                  index=False)

3

In [34]:
#Connection string for connecting python and SQL
def create_connection_string():
    schema = "gans"
    host = "127.0.0.1"
    user = "root"
    password = sql_root_passwort
    port = 3306
    return f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [42]:
connection_string = create_connection_string()


country_from_sql=pd.read_sql('country',con=connection_string)
country_from_sql

,country_id,country_name
0,1,Germany
1,2,France
2,3,Netherlands


### Connecting city data to SQL

In [50]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from lat_lon_parser import parse  #to convert to decimal

def citydetails(cities):

    citydetail=[]
    
    for city in cities:
    #Load html
      url=f'https://en.wikipedia.org/wiki/{city}'
      response=requests.get(url)
      city_soup=BeautifulSoup(response.content,'html.parser')
      city_latitude=city_soup.find(class_='latitude').get_text()
      city_longitude=city_soup.find(class_='longitude').get_text()
      city_country=city_soup.find(class_="infobox-data").get_text() 
      citydetail.append({'city':city,
                         'longitude':parse(city_longitude),
                         'latitude':parse(city_latitude),
                           'country':city_country})
    
    return pd.DataFrame(citydetail)

In [52]:
cities=['Berlin','Paris','Hamburg']
citydata=citydetails(cities)
citydata

,city,longitude,latitude,country
0,Berlin,13.405000,52.520000,Germany
1,Paris,2.352222,48.856667,France
2,Hamburg,10.000000,53.550000,Germany


In [55]:
city=citydata.merge(country_from_sql,
                   left_on="country", right_on="country_name",
                   how='left')

In [65]:
city.drop(columns=["country","country_name"],inplace=True)

In [67]:
city

,city,longitude,latitude,country_id
0,Berlin,13.405000,52.520000,1
1,Paris,2.352222,48.856667,2
2,Hamburg,10.000000,53.550000,1


### Connecting city table to SQL

In [70]:
city.to_sql('city',
                  if_exists='append',
                  con=connection_string,
                  index=False)

3

### Creating table with population

In [128]:
from datetime import datetime # to get today's date
import pandas as pd
import requests
from bs4 import BeautifulSoup

def city_population(cities):

# create one single list to keep track of all values (instead of having separate lists)
    population_data = []

    for city in cities:
        url = f"https://www.wikipedia.org/wiki/{city}"
        response = requests.get(url)
        city_soup = BeautifulSoup(response.content, 'html.parser')

    # extract the relevant information
        city_population = city_soup.find(string="Population").find_next("td").get_text()
        city_population_clean = city_population.replace(",", "")
        today = datetime.today().strftime("%d.%m.%Y")

    # for each city we append a dictionary of values to the list
        population_data.append({"city": city,
                     "population": int(city_population_clean),
                     "population_Timestamp": today
                    })

    population_data = pd.DataFrame(population_data)
    return population_data

In [130]:
city_from_sql=pd.read_sql('city',con=connection_string)

In [132]:
city_from_sql

,city_id,city,longitude,latitude,country_id
0,1,Berlin,13.40500,52.5200,1
1,2,Paris,2.35222,48.8567,2
2,3,Hamburg,10.00000,53.5500,1


In [134]:
cities=city_from_sql['city']
city_popul=city_population(cities)

In [136]:
city_popul

,city,population,population_Timestamp
0,Berlin,3596999,20.03.2025
1,Paris,2048472,20.03.2025
2,Hamburg,1964021,20.03.2025


In [138]:
city_population=city_popul.merge(city_from_sql,
                                on='city',
                                how='left')

In [140]:
city_population.drop(columns=['longitude','latitude','country_id'],inplace=True)

In [142]:
city_population

,city,population,population_Timestamp,city_id
0,Berlin,3596999,20.03.2025,1
1,Paris,2048472,20.03.2025,2
2,Hamburg,1964021,20.03.2025,3


In [146]:
city_population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   city                  3 non-null      object
 1   population            3 non-null      int64 
 2   population_Timestamp  3 non-null      object
 3   city_id               3 non-null      int64 
dtypes: int64(2), object(2)
memory usage: 228.0+ bytes


In [160]:
city_population['population_Timestamp']=pd.to_datetime(city_population['population_Timestamp'])

In [162]:
city_population.to_sql('city_population',
                  if_exists='append',
                  con=connection_string,
                  index=False)

3